# Part 3 – Quantitative Risk Analysis


1. Calculate the 14-day historical 99% VaR for the stETH/ETH basis using a 720 day lookback window.
2. Calculate rolling VaR
3. Visualize rolling VaR and stETH/ETH basis
4. Brief comment on plot, market dynamics and events, implications for liquidity management.

Note: Plots in this notebook are produced with plotly and rendered in browser, change or delete "pio.renderers.default = "browser"" to change this behaviour!

## Load Modules

In [10]:
import pandas as pd
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio

# --- Plotly renderer: open in browser ---
pio.renderers.default = "browser"

from lido_takehome.risk import (
    compute_steth_eth_basis,
    compute_14d_change,
    compute_rolling_var,
)

from dotenv import load_dotenv
from lido_takehome.market_data import fetch_dune_query_results


## Fetch Data from Dune Query

In [11]:
load_dotenv()

DUNE_API_KEY = os.getenv("DUNE_API_KEY")
QUERY_ID = int(os.getenv("DUNE_QUERY_ID"))

df = fetch_dune_query_results(QUERY_ID, DUNE_API_KEY)
df.head()

,date,steth_price_usd,eth_price_usd
0,2021-08-26 00:00:00+00:00,3081.740625,3118.744444
1,2021-08-27 00:00:00+00:00,3119.665382,3159.680000
2,2021-08-28 00:00:00+00:00,3152.351042,3238.580451
3,2021-08-29 00:00:00+00:00,3112.989028,3252.177813
4,2021-08-30 00:00:00+00:00,3069.460208,3230.866528


In [19]:
# Quick Sanity Check on the raw price data

fig = px.line(
    df,
    x="date",
    y=["eth_price_usd", "steth_price_usd"],
    labels={
        "value": "Price (USD)",
        "variable": "Asset",
        "date": "Date",
    },
    title="ETH vs stETH USD Prices (Dune data)",
)
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, x=0), template="plotly_white")
fig.show()

## Calculations

### Compute stETH to ETH Basis and 14-day returns

In [13]:
# 1) basis (deviation from par)
df = compute_steth_eth_basis(
    df,
    steth_col="steth_price_usd",
    eth_col="eth_price_usd",
    as_deviation_from_par=True,
)

In [14]:
# 2) 14-day change in basis (not a pct-change)
df = compute_14d_change(
    df,
    col="basis",
    out_col="basis_change_14d",
)


### Compute rolling Value-at-Risk (99% over 720 lookback for 14-day returns)

In [15]:
# 3) 720-day rolling 99% VaR on that change
df = compute_rolling_var(
    df,
    col="basis_change_14d",
    window=720,
    quantile=0.01,
    out_col="basis_change_14d_var_720d_1p",
)

In [16]:
df.tail()

,date,steth_price_usd,eth_price_usd,basis,basis_change_14d,basis_change_14d_var_720d_1p
1563,2025-12-06 00:00:00+00:00,3036.937986,3038.538889,-0.000527,0.000678,-0.003393
1564,2025-12-07 00:00:00+00:00,3053.321806,3054.056146,-0.000240,0.000743,-0.003393
1565,2025-12-08 00:00:00+00:00,3124.349479,3125.965451,-0.000517,0.000350,-0.003393
1566,2025-12-09 00:00:00+00:00,3189.962118,3193.043194,-0.000965,-0.000010,-0.003393
1567,2025-12-10 00:00:00+00:00,3314.571471,3318.909941,-0.001307,-0.000170,-0.003393


## Plot the VaR and the stETH to ETH Basis

In [20]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["basis"] * 100,
        name="stETH/ETH basis (%, stETH/ETH - 1)",
        hovertemplate="Date: %{x}<br>Basis: %{y:.3f}%<extra></extra>"
    ),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=df["date"],
        y=-df["basis_change_14d_var_720d_1p"] * 100,  # make VaR positive
        name="14d 99% VaR of basis change (%, 720d lookback)",
        hovertemplate="Date: %{x}<br>VaR: %{y:.3f}%<extra></extra>"
    ),
    secondary_y=False,
)

fig.update_layout(
    title="stETH/ETH Basis and 14-Day 99% Historical VaR (720-day lookback)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    template="plotly_white",  xaxis_title="Date"
)

fig.update_yaxes(title_text="Basis (%)", secondary_y=True, showgrid=False)
fig.update_yaxes(title_text="VaR (%)", secondary_y=False)

fig.show()

## Brief Commentary on Market, Events, Liquidity Implications

- The stETH/ETH basis shows clear stress episodes in mid-2022 during the Celsius and Three Arrows Capital unwind, visible as sharp temporary discounts of stETH versus ETH.
- Furthermore, we see a depeg and a lot of volatility in late summer of 2021.
- Because the VaR uses a long 720-day historical window, these extreme tail movements remain in the dataset for two years, keeping the 14-day 99% VaR elevated even after the basis stabilizes following the activation of withdrawals in 2023. 
- The sharp decline in VaR in mid-2024 reflects the point at which these crisis-era data points finally fall out of the rolling window, leaving only the recent, highly stable peg dynamics and therefore implying substantially lower tail-risk in today’s stETH/ETH market.
- For liquidity planning, the drop in VaR suggests reduced immediate tail-risk, but the 2022 episode shows that liquidity buffers should anticipate rare yet severe dislocations.

Comments regarding the plot:

- Two-axis charts can be misleading if used carelessly. Here, the primary focus is the VaR series; the basis is shown mainly for context.
- For that reason, VaR is plotted on the primary (left) axis and drives the grid lines, while the basis is on the secondary (right) axis.
- Since there is more "action" on the LHS for the basis and more action on the RHS for the VaR flipping the y-axis from left to right (primary vs. secondary axes) could have been legitimate decision as well.
- I show the full history of the basis, even though VaR is missing at the very start (due to rolling window), to make the jump and subsequent decline in VaR more interpretable. The link between depegs and market normalization and VaR drops is directly visible.
- The plot is implemented in Plotly for interactivity and easy sharing as an HTML file; I typically use Matplotlib only for quick exploratory sketches, not “production-level” visuals.
- Tooltip values are rounded (basis to a few bps, VaR to a few tenths of a percent) to keep the display readable.